In [1]:
# 导入爬虫所需的第三方库：
# requests用于发送HTTP请求获取网页内容
# BeautifulSoup用于解析HTML文档提取数据
# csv用于将爬取的数据保存为表格格式
# time用于控制爬取间隔时间
# random用于生成随机数，模拟人类浏览的随机停顿
import requests
from bs4 import BeautifulSoup
import csv
import time
import random


In [2]:
# 定义爬取函数：负责爬取目标网站前20页的房源信息，返回所有房源数据的列表
def crawl_house_info():
    # 初始化空列表，用于存储所有爬取到的房源信息
    # 列表中的每个元素是一个字典，对应一条房源的详细信息（如标题、价格、户型等）
    all_houses = []
    
    # 循环爬取前20页数据：range(1,21)生成1到20的整数，对应第1页到第20页
    for page in range(1, 21):
        # 打印当前爬取的页码，方便监控程序运行进度
        print(f"正在爬取第{page}页...")
        
        # 用f-string格式化字符串，动态拼接页码参数
        url = f"https://sh.esf.fang.com/house-a019-b02768/i3{page}/"
        
        # 配置请求头：模拟浏览器发送请求，避免被网站识别为爬虫而拦截
        headers = {
            # User-Agent：告诉服务器访问的设备和浏览器版本（这里模拟Chrome浏览器）
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
            # Accept：声明客户端可接收的内容格式
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
            # Accept-Language：声明客户端偏好的语言（中文）
            "Accept-Language": "zh-CN,zh;q=0.8,en-US;q=0.5,en;q=0.3",
            # Connection：保持TCP连接，减少重复建立连接的开销
            "Connection": "keep-alive"
        }
        
        # 异常捕获块：处理单页爬取过程中可能出现的错误（如网络中断、页面解析失败等）
        try:
            # 发送GET请求获取网页内容：
            # url为目标页面地址，headers为请求头，allow_redirects=True允许自动跳转（避免重定向错误）
            response = requests.get(url, headers=headers, allow_redirects=True)
            
            # 处理网页编码：解决中文乱码问题
            if response.encoding:
                # 若响应头中包含编码信息，直接使用该编码解析
                html_content = response.text
            else:
                # 若响应头无编码信息，尝试用网站常用的GB2312编码解析
                try:
                    response.encoding = 'gb2312'
                    html_content = response.text
                except UnicodeDecodeError:
                    # 若GB2312解析失败，改用兼容的GBK编码（GBK是GB2312的超集）
                    response.encoding = 'gbk'
                    html_content = response.text
            
            # 创建BeautifulSoup对象：解析HTML内容
            # 使用html.parser解析器（Python内置，无需额外安装）
            soup = BeautifulSoup(html_content, 'html.parser')
            
            # 定位所有房源节点：
            # 根据网页HTML结构，房源信息都包裹在class="clearfix"且dataflag="bg"的<dl>标签中
            house_list = soup.find_all('dl', class_='clearfix', dataflag='bg')
            
            # 遍历每个房源节点，提取单条房源的详细信息
            for house in house_list:
                # 初始化空字典，用于存储当前房源的所有字段信息
                house_info = {}
                
                # 提取房源标题：标题信息在<h4>标签下的<a>标签中
                # 先查找<h4>标签（避免直接链式调用导致NoneType错误）
                h4_tag = house.find('h4')
                if h4_tag:
                    # 再从<h4>标签中查找<a>标签（标题所在的具体标签）
                    title_tag = h4_tag.find('a')
                    # 从<a>标签的title属性中提取标题文本，若缺失则用"无标题"代替，并用strip()去除前后空格
                    house_info['标题'] = title_tag.get('title', '无标题').strip() if title_tag else '无标题'
                else:
                    # 若未找到<h4>标签，标题字段设为"无标题"
                    house_info['标题'] = '无标题'
                
                # -------------------------- 提取核心详情信息（户型/面积/楼层等） --------------------------
                # 核心详情信息在class="tel_shop"的<p>标签中
                detail_tag = house.find('p', class_='tel_shop')
                if detail_tag:
                    # 初始化所有详情字段的默认值为"未知"，避免因字段缺失导致键不存在
                    house_info['户型'] = '未知'
                    house_info['面积'] = '未知'
                    house_info['楼层'] = '未知'
                    house_info['朝向'] = '未知'
                    house_info['建造年份'] = '未知'
                    house_info['经纪人'] = '未知'

                    # 1. 提取楼层信息：楼层在class="link_rk"的<a>标签中
                    floor_tag = detail_tag.find('a', class_='link_rk')
                    if floor_tag:
                        # 先获取<a>标签中的基础楼层文本（如"中层"）
                        floor_text = floor_tag.text.strip()
                        # 查找楼层标签后的兄弟节点（包含总楼层信息，如"（共18层）"）
                        next_sibling = floor_tag.next_sibling
                        # 过滤非文本节点（如换行符、空白符等）
                        while next_sibling and isinstance(next_sibling, str) is False:
                            next_sibling = next_sibling.next_sibling
                        # 若找到总楼层信息，拼接完整楼层文本
                        if next_sibling:
                            floor_text += next_sibling.strip()
                        # 将完整楼层信息存入字典
                        house_info['楼层'] = floor_text

                    # 2. 提取经纪人信息：经纪人在class="people_name"的<span>标签中
                    people_span = detail_tag.find('span', class_='people_name')
                    if people_span:
                        # 经纪人姓名在<span>标签下的<a>标签中
                        agent_tag = people_span.find('a')
                        if agent_tag:
                            # 提取<a>标签中的文本作为经纪人姓名
                            house_info['经纪人'] = agent_tag.text.strip()

                    # 3. 提取其他字段（户型/面积/朝向/建造年份）：通过文本特征匹配
                    # 获取detail_tag内所有纯文本内容（过滤标签，仅保留文字）
                    # stripped_strings会自动去除文本前后的空白，再用列表推导式过滤空文本
                    all_texts = [text.strip() for text in detail_tag.stripped_strings if text.strip()]
                    # 遍历所有文本片段，根据关键词匹配对应字段
                    for text in all_texts:
                        # 匹配户型：无特殊关键词，且不是其他字段（通过排除法）
                        if house_info['户型'] == '未知' and '㎡' not in text and '层' not in text and '向' not in text and '年建' not in text:
                            house_info['户型'] = text
                        # 匹配面积：包含"㎡"符号（如"139.03㎡"）
                        elif '㎡' in text:
                            house_info['面积'] = text
                        # 匹配朝向：包含"向"字（如"南向"、"西北向"）
                        elif '向' in text:
                            house_info['朝向'] = text
                        # 匹配建造年份：包含"年建"关键词（如"2000年建"）
                        elif '年建' in text:
                            house_info['建造年份'] = text
                # ------------------------------------------------------------------------------------------
                
                # 提取小区名称和地址：信息在class="add_shop"的<p>标签中
                add_tag = house.find('p', class_='add_shop')
                if add_tag:
                    # 小区名称在<p>标签下的<a>标签中
                    community_tag = add_tag.find('a')
                    if community_tag:
                        # 从<a>标签的title属性提取小区名称，若缺失则用<a>标签文本代替
                        house_info['小区名称'] = community_tag.get('title', community_tag.text).strip()
                        # 地址为去除小区名称后的剩余文本（如"龙华 天钥桥路968弄"）
                        # 用replace方法移除小区名称，仅保留地址部分
                        address_text = add_tag.text.replace(community_tag.text.strip(), '', 1).strip()
                    else:
                        # 若无<a>标签，小区名称设为"未知"
                        house_info['小区名称'] = '未知'
                        # 地址直接使用<p>标签的文本内容
                        address_text = add_tag.text.strip()
                    # 若地址文本为空，设为"未知"
                    house_info['地址'] = address_text if address_text else '未知'
                
                # 提取房源标签（如"满五"、"近地铁"等）：标签在class="label"的<p>标签中
                label_tag = house.find('p', class_='label')
                if label_tag:
                    # 初始化空列表存储所有标签
                    labels = []
                    # 标签可能在<a>标签或<span>标签中，遍历这两种标签
                    for label in label_tag.find_all(['a', 'span']):
                        # 提取标签文本并去除前后空格
                        label_text = label.text.strip()
                        # 仅添加非空标签
                        if label_text:
                            labels.append(label_text)
                    # 用逗号拼接所有标签，若没有标签则设为"无"
                    house_info['标签'] = ', '.join(labels) if labels else '无'
                else:
                    # 若未找到标签容器，标签字段设为"无"
                    house_info['标签'] = '无'
                
                # 提取价格信息（总价和单价）：价格在class="price_right"的<dd>标签中
                price_tag = house.find('dd', class_='price_right')
                if price_tag:
                    # 提取总价：总价在class="red"的<span>标签中（红色字体突出显示）
                    total_price_tag = price_tag.find('span', class_='red')
                    if total_price_tag:
                        house_info['总价'] = total_price_tag.text.strip()
                    else:
                        house_info['总价'] = '未知'
                    
                    # 提取单价：单价是第2个<span>标签（第1个是总价）
                    unit_price_tags = price_tag.find_all('span')
                    if len(unit_price_tags) > 1:
                        house_info['单价'] = unit_price_tags[1].text.strip()
                    else:
                        house_info['单价'] = '未知'
                
                # 将当前房源的信息字典添加到总列表中
                all_houses.append(house_info)
            
            # 设置随机延迟：每次爬取一页后停顿1-3秒（随机值）
            # 模拟人类浏览网页的节奏，避免频繁请求被网站反爬机制拦截
            time.sleep(random.uniform(1, 3))
            
        # 捕获爬取过程中的所有异常（如网络错误、解析错误等）
        except Exception as e:
            # 打印错误信息，方便排查问题（不终止程序，继续爬取下一页）
            print(f"爬取第{page}页时出错: {str(e)}")
            continue  # 跳过当前错误页，继续循环执行下一页
    
    # 爬取完成后，打印总结果（共获取的房源数量）
    print(f"爬取完成，共获取{len(all_houses)}条房源信息")
    # 返回所有房源数据列表
    return all_houses


In [3]:
# 定义保存函数：将爬取到的房源数据保存为CSV文件（表格格式，便于后续分析）
def save_to_csv(houses, filename='housing_price_data_longhua.csv'):
    # 先判断是否有房源数据，若为空则提示并返回
    if not houses:
        print("没有房源信息可保存")
        return
    
    # 提取所有字段名：确保CSV表头包含所有可能的字段（不同房源可能字段不全）
    fieldnames = set()  # 用集合存储字段名，自动去重
    for house in houses:
        # 将每个房源字典的键（字段名）添加到集合中
        fieldnames.update(house.keys())
    # 将集合转为有序列表（按字母排序，使表头顺序固定）
    fieldnames = sorted(fieldnames)
    
    # 打开CSV文件并写入数据：
    # 'w'表示写入模式，newline=''避免空行，encoding='utf-8-sig'确保中文在Excel中正常显示
    with open(filename, 'w', newline='', encoding='utf-8-sig') as csvfile:
        # 创建CSV写入对象，指定表头字段
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        # 写入表头（第一行，字段名）
        writer.writeheader()
        # 遍历每条房源数据，写入CSV文件（每行对应一条房源）
        for house in houses:
            writer.writerow(house)
    
    # 打印保存结果，提示文件保存路径
    print(f"房源信息已保存到{filename}")


In [4]:
# 主程序入口：当脚本直接运行时执行（被导入为模块时不执行）
if __name__ == "__main__":
    # 1. 调用爬取函数，获取所有房源数据
    house_data = crawl_house_info()
    
    # 2. 若有房源数据，调用保存函数写入CSV文件
    if house_data:
        save_to_csv(house_data)


正在爬取第1页...
正在爬取第2页...
正在爬取第3页...
正在爬取第4页...
正在爬取第5页...
正在爬取第6页...
正在爬取第7页...
正在爬取第8页...
正在爬取第9页...
正在爬取第10页...
正在爬取第11页...
正在爬取第12页...
正在爬取第13页...
正在爬取第14页...
正在爬取第15页...
正在爬取第16页...
正在爬取第17页...
正在爬取第18页...
正在爬取第19页...
正在爬取第20页...
爬取完成，共获取1200条房源信息
房源信息已保存到housing_price_data_longhua.csv
